In [51]:
from functools import partial, wraps
import pickle
import os

In [52]:
def parfum(func=None, *, params={}):
    PERSISTENCE_PATH = 'persistence'
    
    if func is None:
        return partial(parfum, params=params)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # save arguments if it receives them
        if len(args):
            # create new arguments
            new_args = []
            for i in range(len(args)):
                if 'input' in params and i in params['input']:
                    new_args.append(params['input'][i])
                elif args[i] == None:
                    with open('{}/{}.input.{}.pkl'.format(PERSISTENCE_PATH, func.__name__, i), 'rb') as f:
                        new_args.append(pickle.load(f))
                else:
                    new_args.append(args[i])
                    with open('{}/{}.input.{}.pkl'.format(PERSISTENCE_PATH, func.__name__, i), 'wb') as f:
                        pickle.dump(args[i], f)
            ret = func(*new_args, **kwargs)
        # load persistent files if no arguments were given
        else:
            files = []
            # load files in persitence folder
            for file in os.listdir(PERSISTENCE_PATH):
                if os.path.isfile(os.path.join(PERSISTENCE_PATH, file)) and file.startswith('{}.input.'.format(func.__name__)):
                    files.append(os.path.join(PERSISTENCE_PATH, file))
            # create new arguments
            new_args = []
            for file in files:
                with open(file, 'rb') as f:
                    new_args.append(pickle.load(f))
            ret = func(*new_args, **kwargs)
        # save output
        if ret in [list, tuple]:
            for i in range(len(ret)):
                with open('{}/{}.output.{}.pkl'.format(PERSISTENCE_PATH, func.__name__, i), 'wb') as f:
                    pickle.dump(ret[i], f)
                if 'output' in params and i in params['output']:
                    params['output'][i](ret[i])
        else:
            with open('{}/{}.output.{}.pkl'.format(PERSISTENCE_PATH, func.__name__, 0), 'wb') as f:
                pickle.dump(ret, f)
            if 'output' in params and 0 in params['output']:
                    params['output'][0](ret)
        return ret
    return wrapper

In [53]:
@parfum(params={
    'input': {
        0: 6000
    },
    'output': {
        0: lambda x: print('salvo', x)
    },
})
def teste(a, b, c):
    return a+b+c

In [54]:
teste(555,2,3)

salvo 6005


6005

In [55]:
teste(3, None, 5)

salvo 6007


6007